In [1]:
import pandas as pd
import numpy as np
from itertools import chain

from context import community_module
from community_module.community_detection.similarityCommunityDetection import SimilarityCommunityDetection
from community_module.similarity.emotionSimilarity import EmotionSimilarity

In [2]:
# Leemos los ficheros de respuestas de usuarios
users_df = pd.read_csv('../data/MNCN/user_profiles.csv')

# Datos necesarios para los filtros
q103_index = list(range(1,7))
q107a_index = list(range(7, 13))
q107b_index = list(range(13, 16))
q110a_index = list(range(16, 18))
q110b_index = list(range(18, len(users_df.columns)))

# Diccionario de respuestas
answers = {
    'q103_0': 'Reducir el tiempo de la ducha',
    'q103_1': 'Comprar menos ropa',
    'q103_2': 'No usar productos con muchos envases',
    'q103_3': 'Ir a pie a más sitios',
    'q103_4': 'Reducir los residuos que genero',
    'q103_5': 'Reciclar de forma correcta',
    'q107a_0': 'En coche',
    'q107a_1': 'Andando',
    'q107a_2': 'En bicicleta',
    'q107a_3': 'En autobús',
    'q107a_4': 'En metro',
    'q107a_5': 'En patinete',
    'q107b_0': 'Estaría dispuesto a cambiar de medio de transporte',
    'q107b_1': 'No estaría dispuesto a cambiar de medio de transporte',
    'q107b_2': 'Quizá estaría dispuesto a cambiar de medio de transporte',
    'q110a_0': 'Si he tenido una mascota exótica',
    'q110a_1': 'No he tenido una mascota exótica',
    'q110b_0': 'Adoptaria una totuga de florida',
    'q110b_1': 'Adoptaria un perro común',
    'q110b_2': 'Adoptaria un gato común',
    'q110b_3': 'Adoptaria una cotorra argentina',
    'q110b_4': 'Adoptaria una cacatúa',
    'q110b_5': 'Adoptaria un mono capuchino',
}

# Funcion de similitud y n clusters a detectar
sim = 'cosine'

In [3]:
def search_groups(data, indexes):
    # Comprobamos que no hay algún usuario que solo contenga ceros
    correct_data = data[data.apply(lambda x: x.values.sum() != 0, axis=1)]

    n_clusters = 2
    finish_search = False

    while not finish_search:

        #print("Clusters:", n_clusters)

        community_detection = SimilarityCommunityDetection(correct_data)
        result = community_detection.calculate_communities(metric=sim, n_clusters=n_clusters)

        correct_data['group'] = result.values()

        # Comprobamos que para cada grupo tenemos al menos, una respuesta común
        answers_per_group_df = correct_data.groupby(by='group').sum().reset_index()
        max_answers_per_group = answers_per_group_df.apply(lambda x: np.max(x[1:].values), axis=1)

        #print("Max answers:", max_answers_per_group.values)

        members_per_group = correct_data.groupby(by='group').count().iloc[:,0].values

        finish_search = np.array_equal(max_answers_per_group, members_per_group)

        #print("Members:", members_per_group)

        if not finish_search:
            n_clusters += 1

    # Genero un DF para extraer las explicaciones de los grupos
    groups_to_explain_df = correct_data.groupby(by='group').sum().reset_index()
    groups_to_explain_df['members'] = members_per_group

    # Genero la tabla de usuario/grupo
    result_df = pd.DataFrame.from_dict(result, orient='index', columns=['group'])
    users_group = users_df.join(result_df)
    
    return groups_to_explain_df, users_group

In [4]:
def explain_groups(groups_to_explain_df):
    for i in range(len(groups_to_explain_df)):

        row = groups_to_explain_df.iloc[i]
        max_value = row[1:-1].max()

        row_ans = row[1:-1]
        row_ans = row_ans[row_ans == max_value]

        print("-----------")
        print("## GROUP", i)
        print("# N. MEMBERS:", int(groups_to_explain_df.iloc[i]['members']))
        print("# COMMON ANS:")

        for i in row_ans.index:
            print("  -", answers[i])

## Buscamos grupos usando únicamente q103

In [5]:
# Paso 1: Definimos los datos a usar para detectar usuarios
indexes = chain(q103_index) # Se pueden combinar
data = users_df.iloc[:,indexes]

In [6]:
groups_to_explain_df, users_groups_df = search_groups(data, indexes)

/Users/jljorro/.pyenv/versions/3.9.6/envs/community-model/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/jljorro/.pyenv/versions/3.9.6/envs/community-model/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/jljorro/.pyenv/versions/3.9.6/envs/community-model/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A 

In [7]:
explain_groups(groups_to_explain_df)

-----------
## GROUP 0
# N. MEMBERS: 3
# COMMON ANS:
  - Comprar menos ropa
-----------
## GROUP 1
# N. MEMBERS: 6
# COMMON ANS:
  - Reducir el tiempo de la ducha
  - No usar productos con muchos envases
  - Ir a pie a más sitios
-----------
## GROUP 2
# N. MEMBERS: 2
# COMMON ANS:
  - Reciclar de forma correcta
-----------
## GROUP 3
# N. MEMBERS: 12
# COMMON ANS:
  - Reducir el tiempo de la ducha
  - No usar productos con muchos envases
  - Reciclar de forma correcta
-----------
## GROUP 4
# N. MEMBERS: 10
# COMMON ANS:
  - Reducir el tiempo de la ducha
  - Ir a pie a más sitios
  - Reciclar de forma correcta
-----------
## GROUP 5
# N. MEMBERS: 1
# COMMON ANS:
  - Ir a pie a más sitios
  - Reciclar de forma correcta
-----------
## GROUP 6
# N. MEMBERS: 2
# COMMON ANS:
  - Reducir el tiempo de la ducha
  - Comprar menos ropa
  - Ir a pie a más sitios
-----------
## GROUP 7
# N. MEMBERS: 1
# COMMON ANS:
  - No usar productos con muchos envases
  - Ir a pie a más sitios
-----------
## 

## Buscamos grupos con las respuestas 107

In [9]:
# Paso 1: Definimos los datos a usar para detectar usuarios
indexes = chain(q107a_index, q107b_index) # Se pueden combinar
data = users_df.iloc[:,indexes]

groups_to_explain_df, users_groups_df = search_groups(data, indexes)

explain_groups(groups_to_explain_df)

-----------
## GROUP 0
# N. MEMBERS: 9
# COMMON ANS:
  - Andando
-----------
## GROUP 1
# N. MEMBERS: 5
# COMMON ANS:
  - En coche
  - Quizá estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 2
# N. MEMBERS: 29
# COMMON ANS:
  - Estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 3
# N. MEMBERS: 1
# COMMON ANS:
  - En patinete
  - Quizá estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 4
# N. MEMBERS: 11
# COMMON ANS:
  - En coche
  - Estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 5
# N. MEMBERS: 3
# COMMON ANS:
  - En autobús
  - No estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 6
# N. MEMBERS: 3
# COMMON ANS:
  - En autobús
  - Quizá estaría dispuesto a cambiar de medio de transporte


## Buscamos grupos con las respuestas 110

In [12]:
# Paso 1: Definimos los datos a usar para detectar usuarios
indexes = chain(q110a_index, q110b_index) # Se pueden combinar
data = users_df.iloc[:,indexes]

groups_to_explain_df, users_groups_df = search_groups(data, indexes)

explain_groups(groups_to_explain_df)

-----------
## GROUP 0
# N. MEMBERS: 40
# COMMON ANS:
  - No he tenido una mascota exótica
-----------
## GROUP 1
# N. MEMBERS: 21
# COMMON ANS:
  - Si he tenido una mascota exótica


## Buscamos grupos con las respuestas 103 y 107

In [14]:
# Paso 1: Definimos los datos a usar para detectar usuarios
indexes = chain(q103_index, q107a_index, q107b_index) # Se pueden combinar
data = users_df.iloc[:,indexes]

groups_to_explain_df, users_groups_df = search_groups(data, indexes)

explain_groups(groups_to_explain_df)

-----------
## GROUP 0
# N. MEMBERS: 3
# COMMON ANS:
  - Reducir el tiempo de la ducha
  - Reducir los residuos que genero
  - Estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 1
# N. MEMBERS: 2
# COMMON ANS:
  - Reducir el tiempo de la ducha
  - Ir a pie a más sitios
  - Reciclar de forma correcta
  - En coche
  - Quizá estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 2
# N. MEMBERS: 4
# COMMON ANS:
  - Reducir los residuos que genero
  - Reciclar de forma correcta
  - No estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 3
# N. MEMBERS: 2
# COMMON ANS:
  - Reducir el tiempo de la ducha
  - Ir a pie a más sitios
  - Reciclar de forma correcta
  - Andando
  - Estaría dispuesto a cambiar de medio de transporte
-----------
## GROUP 4
# N. MEMBERS: 2
# COMMON ANS:
  - Reducir el tiempo de la ducha
  - No usar productos con muchos envases
  - Reciclar de forma correcta
  - Andando
  - Quizá estaría dispuesto a cambiar de med